## Extreme Learning Machine to Multiclass Classification

### Libraries

In [1]:
import numpy as np
import pandas as pd

### Functions

In [2]:
def one_hot_encoding(targets, n_classes):
    return np.eye(n_classes)[targets] # one-hot encoding

In [3]:
def revert_one_hot_encoding(y):
    return np.argmax(y, axis=1) # axis=1 means that we want to find the index of the maximum value in each row

In [4]:
def split_data(X, y, test_size, random_state=None):
    
    np.random.seed(seed=random_state) # set random seed

    indices = np.arange(X.shape[0]) # arange the indices
    np.random.shuffle(indices) # shuffle the indices
    
    X = X[indices] # assign the shuffled X
    y = y[indices] # assign the shuffled y

    n_test = int(test_size * X.shape[0]) # calculate the number of test samples
    
    return X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:] # return the train and test data

In [5]:
def feature_normalize(X):
    
    mu = np.mean(X, axis=0) # Mean of each feature
    sigma = np.std(X, axis=0) # Standard deviation of each feature
    
    X_norm = (X-mu)/sigma # Normalized data (zero mean and unite standard deviation)
    
    return X_norm, mu, sigma

In [6]:
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred) # mean of the elements that are equal

In [7]:
def confusion_matrix(y_true, y_pred):

    n_classes = len(np.unique(y_true)) # Number of classes

    cm = np.zeros((n_classes, n_classes)) # initialize the confusion matrix
    
    for i in range(len(y_true)): # for each sample
        cm[y_true[i], y_pred[i]] += 1 # add 1 to the corresponding row and column
    
    return cm

In [8]:
def elm_train(X, y, L, w1=None):
    
    M = np.size(X, axis=0) # Number of examples
    N = np.size(X, axis=1) # Number of features
    
    # If w1 is not defined
    if w1 is None:
        #w1 = np.random.rand(L, N+1);  # Weights with bias
        w1 = np.random.uniform(low=-1, high=1, size=(L, N+1)) # Weights with bias

    bias = np.ones(M).reshape(-1, 1) # Bias definition
    #bias.shape = (M, 1)
    Xa = np.concatenate((bias, X), axis=1) # Input with bias

    S = Xa.dot(w1.T) # Weighted sum of hidden layer
    H = np.tanh(S) # Activation function f(x) = tanh(x), dimension M X L

    bias = np.ones(M).reshape(-1, 1) # Bias definition
    #bias.shape = (M, 1)
    Ha = np.concatenate((bias, H), axis=1) # Activation function with bias

    w2 = (np.linalg.pinv(Ha).dot(y)).T # w2' = pinv(Ha)*D

    y_pred = Ha.dot(w2.T) # Predictions
    
    return y_pred, w1, w2

In [9]:
def elm_test(X, w1, w2):
    
    M = np.size(X, axis=0) # Number of examples
    N = np.size(X, axis=1) # Number of features

    bias = np.ones(M).reshape(-1, 1) # Bias definition
    #bias.shape = (M, 1)
    Xa = np.concatenate((bias, X), axis=1) # Input with bias

    S = Xa.dot(w1.T) # Weighted sum of hidden layer
    H = np.tanh(S) # Activation function f(x) = tanh(x), dimension M X L

    bias = np.ones(M).reshape(-1, 1) # Bias definition
    #bias.shape = (M, 1)
    Ha = np.concatenate((bias, H), axis=1) # Activation function with bias

    y_pred = Ha.dot(w2.T) # Predictions
    
    return y_pred

### Iris dataset

In [11]:
df = pd.read_csv('../data/Iris.csv', sep=',')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Adjust the target: one-hot enconding

In [12]:
target_column = 'Species' # Target column
specie_list = np.unique(df[target_column]) # Species list

print('Species:', specie_list)

Species: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [13]:
species_range = np.arange(0, len(specie_list)) # Species range
print('Species range:', species_range)

i=0
for specie in specie_list:
    
    df[target_column].replace(specie, species_range[i], inplace=True) # Replace species with range

    i+=1

df.head()

Species range: [0 1 2]


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [14]:
targets = df[target_column].values # Targets
n_classes = len(np.unique(targets)) # Number of classes

y = one_hot_encoding(targets, n_classes) # One-hot encoding

print('Old target:', targets[0])
print('New target:', y[0])

Old target: 0
New target: [1. 0. 0.]


### Training

Select the features

In [27]:
X = df.drop(labels=['Id', target_column], axis=1).values # Features

Train-test split

In [44]:
X_train, X_test, y_train, y_test = split_data(X, y, 0.2) # Split data

print('Train data shape:', X_train.shape)
print('Teste data shape:', X_test.shape)

Train data shape: (120, 4)
Teste data shape: (30, 4)


Normalization

In [45]:
X_train, mu, sigma = feature_normalize(X_train) # Normalize features

X_test = (X_test-mu)/sigma # Normalized data (zero mean and unite standard deviation)

print('Train mean:', np.mean(X_train, axis=0))
print('Train std:', np.std(X_train, axis=0))
print('Test mean:', np.mean(X_test, axis=0))
print('Test std:', np.std(X_test, axis=0))

Train mean: [-2.04095999e-15  7.41999055e-16 -6.21724894e-16  3.57584333e-16]
Train std: [1. 1. 1. 1.]
Test mean: [ 0.04994619  0.05348415 -0.01066048  0.02917204]
Test std: [0.9442107  0.75578623 0.88500103 0.92837925]


Training the model

In [18]:
L = 15 # Number of hidden neurons

print('Number of hidden neuros:', L)

y_train_pred, w1, w2 = elm_train(X_train, y_train, L=L) # Train
y_test_pred = elm_test(X_test, w1, w2) # Test

Number of hidden neuros: 15


Training the model: predefined weights

In [46]:
data = np.load('../models/clf-weights.npz')
w1 = data['w1']

L = w1.shape[0] # Number of hidden neurons

print('Number of hidden neuros:', L)

y_train_pred, w1, w2 = elm_train(X_train, y_train, L, w1=w1)
y_test_pred = elm_test(X_test, w1, w2)

Number of hidden neuros: 15


Accuracy

In [47]:
train_acc = accuracy(revert_one_hot_encoding(y_train), revert_one_hot_encoding(y_train_pred))
test_acc = accuracy(revert_one_hot_encoding(y_test), revert_one_hot_encoding(y_test_pred))

print('Training accuracy (%):', train_acc*100)
print('Testing accuracy (%):', test_acc*100)

Training accuracy (%): 98.33333333333333
Testing accuracy (%): 96.66666666666667


Confusion matrix

In [48]:
confusion_matrix(revert_one_hot_encoding(y_test), revert_one_hot_encoding(y_test_pred))

array([[ 9.,  0.,  0.],
       [ 0., 14.,  1.],
       [ 0.,  0.,  6.]])

### Single classification

Get the data

In [49]:
data = df.drop(labels=['Id'], axis=1)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [50]:
i = np.random.randint(0, len(data)) # select a random example

x_data = data.drop(labels=[target_column], axis=1).iloc[i].values
y_label = data[target_column].iloc[i]

print('Single example:', data.iloc[i])

Single example: SepalLengthCm    5.4
SepalWidthCm     3.7
PetalLengthCm    1.5
PetalWidthCm     0.2
Species          0.0
Name: 10, dtype: float64


In [51]:
print('Data:', x_data)
print('Target = %d and class = %s' % (y_label, specie_list[y_label]))

Data: [5.4 3.7 1.5 0.2]
Target = 0 and class = Iris-setosa


Normalization

In [52]:
x_data = x_data.reshape(1, -1) # Reshape data

x_data = (x_data-mu)/sigma # Normalized data (zero mean and unite standard deviation)

One-hot encoding

In [53]:
y_data = one_hot_encoding(y_label, n_classes)
y_data = y_data.reshape(1, -1)

Prediction

In [54]:
y_data_pred = elm_test(x_data, w1, w2)

print('Class:', y_data)
print('Predicted class:', y_data_pred)

Class: [[1. 0. 0.]]
Predicted class: [[ 0.99993918  0.06712146 -0.06706064]]
